In [88]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.ensemble import GradientBoostingClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import timeit
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from pyspark.sql import SparkSession
import findspark
import os
from pyspark.sql.functions import col
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import math
from pyspark.ml.tuning import CrossValidator







os.environ["JAVA_HOME"] = 'C:/Program Files/Java/jre1.8.0_333/'
os.environ["SPARK_HOME"] = 'C:/spark/spark-3.3.0-bin-hadoop2/'
os.environ["HADOOP_HOME"] = 'C:/hadoop/'
#os.environ["PYSPARK_SUBMIT_ARGS"] = '--master local pyspark-shell'

In [89]:
findspark.find()

'C:/spark/spark-3.3.0-bin-hadoop2/'

In [90]:
findspark.init()

In [91]:
df = pd.read_csv('C:/Users/paulc/Documents/Data Analysis Courses/MastersChurn/ProjectWork/ChurnModelv1.csv')

#kendall_dataframe = pd.read_csv('C:/Users/paulc/Documents/Data Analysis Courses/MastersChurn/ProjectWork/kendall_dataframe.csv')
#anova_dataframe = pd.read_csv('C:/Users/paulc/Documents/Data Analysis Courses/MastersChurn/ProjectWork/anova_dataframe.csv')

In [92]:
# remap target column to be numerical
df['Active_Next30Days_Cash_YN'] = df['Active_Next30Days_Cash_YN'].replace({"no":0,"yes":1})


In [93]:
df.columns

Index(['MonthNumber', 'ClientProfileSummary', 'Gender', 'CashActive_YN',
       'L1DSBBetCount', 'L7DSBBetCount', 'L30DSBBetCount', 'L90DSBBetCount',
       'L1DDepositCount', 'L7DDepositCount', 'L30DDepositCount',
       'L90DDepositCount', 'L7DOtherSportsBetCount', 'L30DOtherSportsBetCount',
       'L90DOtherSportsBetCount', 'L90DUnsuccessfulDepositCount',
       'DaysSinceLastSBCashAPD', 'DaysSinceLastSBAPD', 'OlderMale40',
       'CustomerConcession90days', 'Active_Next30Days_Cash_YN',
       'L7DSBTurnover', 'US_SportsTurnover7D', 'L30DSBTurnover',
       'US_SportsTurnover30D', 'CustomerConcession30days', 'L90DSBTurnover',
       'L1DSBTurnover', 'US_SportsTurnover1D', 'L30DSBFreeBetsHandle',
       'US_SportsTurnover90D', 'L7DSBFreeBetsHandle'],
      dtype='object')

<h3> Pyspark for modelling

In [94]:
spark = SparkSession\
.builder\
.master("local")\
.config("spark.executor.cores", "4")\
.config("spark.executor.memory", "10g")\
.config("spark.driver.memory", "10g")\
.config("spark.executor.memoryOverhead", "10g")\
.appName("Churn Project").getOrCreate()

# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [8]:
df = spark.createDataFrame(df)

In [9]:
#df.write.parquet("/tmp/Churn/kendall.parquet")

First, I have used Vector Assembler to combine predictor columns into a single vector column. Here the new single vector column is called features.



In [10]:
numericCols = ['MonthNumber', 'ClientProfileSummary', 'Gender', 'CashActive_YN',
       'L1DSBBetCount', 'L7DSBBetCount', 'L30DSBBetCount', 'L90DSBBetCount',
       'L1DDepositCount', 'L7DDepositCount', 'L30DDepositCount',
       'L90DDepositCount', 'L7DOtherSportsBetCount', 'L30DOtherSportsBetCount',
       'L90DOtherSportsBetCount', 'L90DUnsuccessfulDepositCount',
       'DaysSinceLastSBCashAPD', 'DaysSinceLastSBAPD', 'OlderMale40',
       'CustomerConcession90days',
       'L7DSBTurnover', 'US_SportsTurnover7D', 'L30DSBTurnover',
       'US_SportsTurnover30D', 'CustomerConcession30days', 'L90DSBTurnover',
       'L1DSBTurnover', 'US_SportsTurnover1D', 'L30DSBFreeBetsHandle',
       'US_SportsTurnover90D', 'L7DSBFreeBetsHandle']

In [11]:
assembler = VectorAssembler(inputCols=numericCols, outputCol="features")
df = assembler.transform(df)

Train/Test Split

In [12]:
train,test = df.randomSplit([0.7,0.3],seed=42)

In [13]:
train.printSchema()

root
 |-- MonthNumber: long (nullable = true)
 |-- ClientProfileSummary: long (nullable = true)
 |-- Gender: long (nullable = true)
 |-- CashActive_YN: long (nullable = true)
 |-- L1DSBBetCount: long (nullable = true)
 |-- L7DSBBetCount: long (nullable = true)
 |-- L30DSBBetCount: long (nullable = true)
 |-- L90DSBBetCount: long (nullable = true)
 |-- L1DDepositCount: long (nullable = true)
 |-- L7DDepositCount: long (nullable = true)
 |-- L30DDepositCount: long (nullable = true)
 |-- L90DDepositCount: long (nullable = true)
 |-- L7DOtherSportsBetCount: long (nullable = true)
 |-- L30DOtherSportsBetCount: long (nullable = true)
 |-- L90DOtherSportsBetCount: long (nullable = true)
 |-- L90DUnsuccessfulDepositCount: long (nullable = true)
 |-- DaysSinceLastSBCashAPD: long (nullable = true)
 |-- DaysSinceLastSBAPD: long (nullable = true)
 |-- OlderMale40: double (nullable = true)
 |-- CustomerConcession90days: double (nullable = true)
 |-- Active_Next30Days_Cash_YN: long (nullable = true)

<H3> Run a base RandomForrest to benchmark </h3>
How Many Trees in a Random Forest? - AU  - Oshiro, Thais
AU  - Perez, Pedro
AU  - Baranauskas, José

Therefore, it is possible to suggest, based on the experiments, a range between 64 and 128 trees in a forest.With these numbers of trees
it is possible to obtain a good balance between AUC, processing time, and memory
usage

In [31]:
rf = RandomForestClassifier(featuresCol='features',labelCol='Active_Next30Days_Cash_YN',numTrees=128,seed=42)


rfModel = rf.fit(train)
predictions = rfModel.transform(test)
predictions.select('prediction','Active_Next30Days_Cash_YN', 'probability').show(25)


+----------+-------------------------+--------------------+
|prediction|Active_Next30Days_Cash_YN|         probability|
+----------+-------------------------+--------------------+
|       0.0|                        0|[0.89965648379616...|
|       0.0|                        0|[0.90143338079076...|
|       0.0|                        0|[0.90199779618197...|
|       0.0|                        0|[0.90199779618197...|
|       0.0|                        0|[0.90199779618197...|
|       0.0|                        0|[0.90199779618197...|
|       0.0|                        0|[0.90199779618197...|
|       0.0|                        0|[0.89965648379616...|
|       0.0|                        0|[0.90143338079076...|
|       0.0|                        0|[0.90199779618197...|
|       0.0|                        0|[0.90199779618197...|
|       0.0|                        0|[0.90199779618197...|
|       0.0|                        0|[0.90199779618197...|
|       0.0|                        0|[0

In [32]:
evaluation = MulticlassClassificationEvaluator(labelCol='Active_Next30Days_Cash_YN', predictionCol='prediction',metricName='accuracy')
accuracy = evaluation.evaluate(predictions)
print("The base RandomForrestClassifier Test error is:", round((1.0 - accuracy),4))

The base RandomForrestClassifier Test error is: 0.1706


In [33]:
print("The Accuracy of the base RandomForrestClassifier is:", round(accuracy,4))

# AUTHOR: I reran the above model with 1000 trees and the resulting accuracy increased to 0.8295 a negligible difference.

The Accuracy of the base RandomForrestClassifier is: 0.8294


In [27]:
rfModel.featureImportances

SparseVector(31, {0: 0.0003, 1: 0.0073, 2: 0.0, 3: 0.0038, 4: 0.0014, 5: 0.1335, 6: 0.0698, 7: 0.048, 8: 0.0, 9: 0.0002, 10: 0.0006, 11: 0.0014, 12: 0.0006, 13: 0.005, 14: 0.0096, 15: 0.0, 16: 0.2129, 17: 0.1541, 18: 0.0, 19: 0.0002, 20: 0.1332, 21: 0.0413, 22: 0.0901, 23: 0.0297, 24: 0.0041, 25: 0.0308, 26: 0.0028, 27: 0.0044, 28: 0.0033, 29: 0.0101, 30: 0.0013})

<h4> Test Results on scaled dataframe

In [42]:
scaled_df = pd.read_csv('C:/Users/paulc/Documents/Data Analysis Courses/MastersChurn/ProjectWork/ChurnModelScaled.csv')

In [43]:
scaled_df = spark.createDataFrame(scaled_df)

In [44]:
numericCols = ['MonthNumber', 'ClientProfileSummary', 'Gender', 'CashActive_YN',
       'L1DSBBetCount', 'L7DSBBetCount', 'L30DSBBetCount', 'L90DSBBetCount',
       'L1DDepositCount', 'L7DDepositCount', 'L30DDepositCount',
       'L90DDepositCount', 'L7DOtherSportsBetCount', 'L30DOtherSportsBetCount',
       'L90DOtherSportsBetCount', 'L90DUnsuccessfulDepositCount',
       'DaysSinceLastSBCashAPD', 'DaysSinceLastSBAPD', 'OlderMale40',
       'CustomerConcession90days',
       'L7DSBTurnover', 'US_SportsTurnover7D', 'L30DSBTurnover',
       'US_SportsTurnover30D', 'CustomerConcession30days', 'L90DSBTurnover',
       'L1DSBTurnover', 'US_SportsTurnover1D', 'L30DSBFreeBetsHandle',
       'US_SportsTurnover90D', 'L7DSBFreeBetsHandle']

In [45]:
scaled_assembler = VectorAssembler(inputCols=numericCols, outputCol="features")
scaled_df = assembler.transform(scaled_df)

In [46]:
train_scaled,test_scaled = scaled_df.randomSplit([0.7,0.3],seed=42)

In [64]:
rfs = RandomForestClassifier(featuresCol='features',labelCol='Active_Next30Days_Cash_YN',numTrees=128,seed=42)


rfsModel = rfs.fit(train_scaled)
predictions_scaled = rfsModel.transform(test_scaled)
predictions_scaled.select('prediction','Active_Next30Days_Cash_YN', 'probability').show(25)

+----------+-------------------------+--------------------+
|prediction|Active_Next30Days_Cash_YN|         probability|
+----------+-------------------------+--------------------+
|       1.0|                      1.0|[0.16496901934784...|
|       0.0|                      0.0|[0.88249680294115...|
|       1.0|                      0.0|[0.41304392482978...|
|       1.0|                      1.0|[0.11282856023456...|
|       1.0|                      1.0|[0.29782286664513...|
|       1.0|                      0.0|[0.49743765676353...|
|       1.0|                      0.0|[0.19037548091788...|
|       0.0|                      0.0|[0.90046207110125...|
|       1.0|                      1.0|[0.11498740368851...|
|       1.0|                      1.0|[0.11498740368851...|
|       1.0|                      1.0|[0.11498740368851...|
|       0.0|                      0.0|[0.89134634595665...|
|       0.0|                      0.0|[0.89331240835112...|
|       0.0|                      0.0|[0

In [65]:
evaluation_scaled = MulticlassClassificationEvaluator(labelCol='Active_Next30Days_Cash_YN', predictionCol='prediction',metricName='accuracy')
accuracy_scaled = evaluation_scaled.evaluate(predictions_scaled)
print("The base RandomForrestClassifier Test error is:", round((1.0 - accuracy_scaled),4))

The base RandomForrestClassifier Test error is: 0.1703


In [66]:
print("The Accuracy of the base RandomForrestClassifier is:", round(accuracy,4))
print("The Accuracy of the base ScaledRandomForrestClassifier is:", round(accuracy_scaled,4))
print("Improvement as a result of scaling is:", round(0.8297-0.8294,4))

The Accuracy of the base RandomForrestClassifier is: 0.8294
The Accuracy of the base ScaledRandomForrestClassifier is: 0.8297
Improvement as a result of scaling is: 0.0003


In [ ]:
#######################################################################################################################################################################

In [ ]:
#######################################################################################################################################################################

<h3> Randomized Hyperparameter search

In [67]:
class RandomGridBuilder:
  '''Grid builder for random search. Sets up grids for use in CrossValidator in Spark using values randomly sampled from user-provided distributions.
  Distributions should be provided as lambda functions, so that the numbers are generated at call time.

  Parameters:
    num_models: Integer (Python) - number of models to generate hyperparameters for
    seed: Integer (Python) - seed (optional, default is None)

  Returns:
    param_map: list of parameter maps to use in cross validation.

  Example usage:
    from pyspark.ml.classification import LogisticRegression
    lr = LogisticRegression()
    paramGrid = RandomGridBuilder(2)\
               .addDistr(lr.regParam, lambda: np.random.rand()) \
               .addDistr(lr.maxIter, lambda : np.random.randint(10))\
               .build()

    Returns similar output as Spark ML class ParamGridBuilder and can be used in its place. The above paramGrid provides random hyperparameters for 2 models.
    '''

  def __init__(self, num_models, seed=None):
    self._param_grid = {}
    self.num_models = num_models
    self.seed = seed

  def addDistr(self, param, distr_generator):
    '''Add distribution based on dictionary generated by function passed to addDistr.'''

    if 'pyspark.ml.param.Param' in str(type(param)):
      self._param_grid[param] = distr_generator
    else:
      raise TypeError('param must be an instance of Param')

    return self

  def build(self):
    param_map = []
    for n in range(self.num_models):
      if self.seed:
        # Set seeds for both numpy and random in case either is used for the random distribution
        np.random.seed(self.seed + n)
        random.seed(self.seed + n)
      param_dict = {}
      for param, distr in self._param_grid.items():
        param_dict[param] = distr()
      param_map.append(param_dict)

    return param_map

In [80]:
clf = RandomForestClassifier(featuresCol='features',labelCol='Active_Next30Days_Cash_YN')

# Set up cross validation with random search
randomParams = RandomGridBuilder(3)\
                 .addDistr(clf.featureSubsetStrategy, lambda : str(np.random.rand()))\
                 .addDistr(clf.maxDepth, lambda : np.random.choice(2, 24))\
                 .addDistr(clf.numTrees, lambda : np.random.choice(20,328))\
                 .addDistr(clf.impurity, lambda : np.random.choice(['gini', 'entropy']))\
                 .build()


ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [ ]:
evaluator = MulticlassClassificationEvaluator()

# Cross validator with random search
cv = CrossValidator(
                    estimator = clf,
                    estimatorParamMaps = randomParams,
                    evaluator = evaluator,
                    parallelism = 2
                   )

cvModel = cv.fit(train)

Random search allowed us to narrow down the range for each hyperparameter. Now that we know where to concentrate our search, we can explicitly specify every combination of settings to try

<h2> GridSearchCV

In [7]:
# use ensemble Random Forest method. First we must perform GridSearchCV to find out the best params.
def model_tuning_GS(model, parameter_dict):
    """Function to perform hyperparameter turning for the classification models using GridSearch."""
    # inspect the model params.
    model.get_params()
    # define the parameters using a dictionary that we want to test.
    model_grid = parameter_dict
    # initialise a GSCV object with the model as an argument. scoring is set to accuracy and CV set to 10.
    Grid_model = GridSearchCV(estimator=model, param_grid=model_grid, cv=10, scoring="accuracy")
    # fit the model to data.
    Grid_model.fit(X_train, y_train)
    # extract the best estimator, accuracy score and print them.
    print("GridSearchCV results:", model.__class__.__name__)
    # print best estimator
    print("Best Estimator:\n", Grid_model.best_estimator_)
    # printing the mean cross-validated score of the best_estimator:
    print("\n Best Score:\n", Grid_model.best_score_)
    # printing the parameter setting that gave the best results on the hold out testing data.:
    print("\n Best Hyperparameters:\n", Grid_model.best_params_)

In [16]:
%timeit
import tqdm

parameter_dict = {'n_estimators': [500, 1000, 3000, 4000, 5000],
                  'max_depth': [100, 200, 300, 400, 500],
                  'min_samples_leaf': [1],
                  'criterion':['gini','entropy'],
                  'bootstrap': [True]}
#bagging

In [17]:
model_tuning_GS(RandomForestClassifier(random_state=42), parameter_dict)
print("Model Trained!")

KeyboardInterrupt: 

In [8]:

# create a parameter dictionary to test for the gradient boosting algorithm.

parameter_dict_GB = {'n_estimators': [100, 200],
                     'subsample': [0.8, 0.4],
                     'max_depth': [1, 5],
                     'learning_rate': [0.01]}

# find the optimal params for GBC
model_tuning_GS(GradientBoostingClassifier(random_state=42), parameter_dict_GB)

NameError: name 'X_train' is not defined

In [ ]:
# perform gradient boosting on optimum params found above.
ensemble = GradientBoostingClassifier(learning_rate=0.01, max_depth=5, n_estimators=200, random_state=42,
                                      subsample=0.8).fit(X_train, y_train)
# make predictions using GDC.
ensemble_predictions = ensemble.predict(X_test)
# print the accuracy, an increase of 3% was found and the model is now 0.89% accurate.
print("Accuracy:", metrics.accuracy_score(y_test, ensemble_predictions).round(decimals=2))